<a href="https://colab.research.google.com/github/ScottHK/cm4706-coursework-2/blob/main/Coursework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Focus Area

The focus of these NLP Pipelines is to apply sentiment analysis to a dataset of patient reviews regarding various drugs they or someone they're responsible for takes for specific conditions. 

The nlp models will then be evaluated to determine which pipeline is better suited to serve the focus area. They will be different, but evaluated in the same way so they can be compared properly.

# Preparation

In [1]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 4.1 MB/s 
     |████████████████████████████████| 106 kB 35.5 MB/s 


In [2]:
import numpy as np
import pandas as pd
from scipy.io.arff import loadarff
from google.colab import files
import io
import contractions
import html
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
import time
from nltk.corpus import stopwords


In [3]:
uploaded = files.upload() # Uploading both Train and Test datasets (drugsComTrain_raw.tsv and drugsComTest_raw.tsv)


Saving drugsComTest_raw.tsv to drugsComTest_raw.tsv
Saving drugsComTrain_raw.tsv to drugsComTrain_raw.tsv


The data has been taken from the following link: https://archive.ics.uci.edu/ml/datasets/Drug+Review+Dataset+%28Drugs.com%29

The dataset describes a drug reviewing websites reviews for specific drugs and conditions.

Attribute Information:

1. drugName (categorical): name of drug
2. condition (categorical): name of condition
3. review (text): patient review
4. rating (numerical): 10 star patient rating
5. date (date): date of review entry
6. usefulCount (numerical): number of users who found review useful

In [22]:
df_train = pd.read_csv('/content/drugsComTrain_raw.tsv', sep='\t')
df_test = pd.read_csv('/content/drugsComTest_raw.tsv', sep='\t')
df_train

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
...,...,...,...,...,...,...,...
161292,191035,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125
161293,127085,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34
161294,187382,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35
161295,47128,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0,"September 19, 2015",79


In [23]:
df_test

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,163740,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",10.0,"February 28, 2012",22
1,206473,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",8.0,"May 17, 2009",17
2,159672,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",9.0,"September 29, 2017",3
3,39293,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",9.0,"March 5, 2017",35
4,97768,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",9.0,"October 22, 2015",4
...,...,...,...,...,...,...,...
53761,159999,Tamoxifen,"Breast Cancer, Prevention","""I have taken Tamoxifen for 5 years. Side effe...",10.0,"September 13, 2014",43
53762,140714,Escitalopram,Anxiety,"""I&#039;ve been taking Lexapro (escitaploprgra...",9.0,"October 8, 2016",11
53763,130945,Levonorgestrel,Birth Control,"""I&#039;m married, 34 years old and I have no ...",8.0,"November 15, 2010",7
53764,47656,Tapentadol,Pain,"""I was prescribed Nucynta for severe neck/shou...",1.0,"November 28, 2011",20


The data contains an additional column that is not explained (Unnamed: 0). Therefore we will drop it.

In [24]:
df_train = df_train.iloc[:, 1:] # Drops the first column
df_test = df_test.iloc[:, 1:]
df_train

,drugName,condition,review,rating,date,usefulCount
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37
...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35
161295,Thyroid desiccated,Underactive Thyroid,"""I&#039;ve been on thyroid medication 49 years...",10.0,"September 19, 2015",79


We require some sentiments in the training data. Applying some logic to the rating system, I surmise that 1-4 is bad, 5-10 is good. This may seem arbitrary, and it very well may be. It seems to me that 5 is the middle ground, but overall more positive. From there and above it would be good, whereas below this threshold it's undeniably negative, therefore bad.

There is also an odd addition of some encoding issues such as &#039 which appears to be an aprostophe. Using the HTML package, this can be resolved using the unescape() function. 

Putting both dataframes into a list cuts down on repeating code by using a for loop.

In [25]:
df_list = [df_train.copy(), df_test.copy()]

for df in df_list:
  df['review'] = df['review'].apply(lambda x: html.unescape(x)) # function takes a string input. Using apply with lambda to apply to series per string

  df['sentiment'] = np.where(df['rating'] < 5, 'Bad', 'Good') # If less than 5 star rating, it's bad. 5 and above it's good.



df_list[0]

,drugName,condition,review,rating,date,usefulCount,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,Good
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,Good
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,Good
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,Good
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,Good
...,...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125,Good
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34,Bad
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35,Bad
161295,Thyroid desiccated,Underactive Thyroid,"""I've been on thyroid medication 49 years, I s...",10.0,"September 19, 2015",79,Good


# NLP Pipeline 1


## Pre-Processing

I will apply contraction expansion and case standardisation for pre-processing.

Starting with **contraction expansion**. Applying the contraction expansion fix() function.

In [26]:
df_list_pre_processing = [df_list[0].copy(), df_list[1].copy()] # List of dfs. 0 is train, 1 is test

for df_pre_processing in df_list_pre_processing: # looping through the dfs in list

  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: contractions.fix(x)) # contractions fix() function takes a string, using apply with lambda to access strings

df_list_pre_processing[0]

,drugName,condition,review,rating,date,usefulCount,sentiment
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,Good
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,Good
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,Good
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,Good
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,Good
...,...,...,...,...,...,...,...
161292,Campral,Alcohol Dependence,"""I wrote my first report in Mid-October of 201...",10.0,"May 31, 2015",125,Good
161293,Metoclopramide,Nausea/Vomiting,"""I was given this in IV before surgey. I immed...",1.0,"November 1, 2011",34,Bad
161294,Orencia,Rheumatoid Arthritis,"""Limited improvement after 4 months, developed...",2.0,"March 15, 2014",35,Bad
161295,Thyroid desiccated,Underactive Thyroid,"""I have been on thyroid medication 49 years, I...",10.0,"September 19, 2015",79,Good


Now to apply **case standardisation**

In [27]:
for df_pre_processing in df_list_pre_processing:
  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: x.lower()) # .lower() requires strings. Using lambda and apply to allow access to df strings

df_list_pre_processing[0]['review'] # Checking the series for anything that jumps out

0         "it has no side effect, i take it in combinati...
1         "my son is halfway through his fourth week of ...
2         "i used to take another oral contraceptive, wh...
3         "this is my first time using any form of birth...
4         "suboxone has completely turned my life around...
                                ...                        
161292    "i wrote my first report in mid-october of 201...
161293    "i was given this in iv before surgey. i immed...
161294    "limited improvement after 4 months, developed...
161295    "i have been on thyroid medication 49 years, i...
161296    "i have had chronic constipation all my adult ...
Name: review, Length: 161297, dtype: object

## Representation Learning

Taking a copy of both dataframes once more, allows for easier rerunning without rerunning the whole of pre=processing.

We're going to use Tf-Idf for the representation learning through skLearn.

In [28]:
df_train = df_list_pre_processing[0].copy() # Can now separate, different functions will be used on both datasets
df_test = df_list_pre_processing[1].copy()

vectorizer = TfidfVectorizer() # Tf-Idf object instantiation
x_train = vectorizer.fit_transform(df_train['review'])
x_test = vectorizer.transform(df_test['review'])



## NLP Model

The aim of this pipeline is to use a Naive Bayes Multinomial model to predict the sentiment in the test dataset. It will do this through probabilities.

In [31]:
NB_classifier = MultinomialNB() # Instantiating the Naive Bayes Multinomial object
NB_classifier_training_start = time.time() # Taking the time for measuring how long training takes
NB_classifier.fit(x_train, df_train['sentiment'])
NB_classifier_training_finish = time.time()
NB_prediction = NB_classifier.predict(x_test) # Predicting the test dataset
NB_classifier_prediction_finish = time.time()



## Evaluation

For the Evaluation stage I will be evaluating the models performance with Precision, Recall, and F1-Score. I will also include the Training and Prediction time, as time as a factor is good at measuring performance. For example, a fast and accurate model may be preferred over a slow and accurate one.

In [32]:
print('Training time: ', NB_classifier_training_finish - NB_classifier_training_start)
print('Prediction time: ', NB_classifier_training_finish - NB_classifier_prediction_finish)
NB_classifier_results = classification_report(df_test['sentiment'], NB_prediction)
print(NB_classifier_results)

Training time:  1.6072309017181396
Prediction time:  -0.02286219596862793
              precision    recall  f1-score   support

         Bad       0.92      0.09      0.17     13497
        Good       0.77      1.00      0.87     40269

    accuracy                           0.77     53766
   macro avg       0.84      0.55      0.52     53766
weighted avg       0.80      0.77      0.69     53766



Looking at the results, the model has a hard time identifying Bad labels. Though precision is high, recall is abysmally low leading to a low f1-score. Conversely, it was exceptionally good at determining good labels.

Overall, the model accuracy came to 77% with a training time of 1.6 seconds. A great result.

# NLP Pipeline 2

## Pre-Processing

For the second pipeline, I'm going to use stop word removal, stemming, contraction expansion, alphanumeric removal, and punctuation removal. Downloading the nltk stopwords corpus:

In [33]:
nltk.download('stopwords') # Downloading NLTK's stopwords corpus

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Creating df list to lower repeated code. Tokenizing the strings, then removing the stopwords and combining the words again separated by a space.

Representation learnings being used are being done on a document basis, instead of a list of lists.

RegexpTokenzier will double up as a tokenizer and a punctuation removal function. Using Regex to target all symbols and numbers. They shouldn't hold much weight so it would be good to remove them.: '\w+|\$[\d\.]+|\S+'

In [34]:
df_list_pre_processing = [df_list[0].copy(), df_list[1].copy()]

english_stopwords = stopwords.words('english') # Accessing the english stopwords specifically

regex_tokenizer = RegexpTokenizer('\w+|\$[\d\.]+|\S+') # Regex tokenizer, using the given regex to target punctuation using regex calculator https://regex101.com/

snowball = SnowballStemmer('english') # Using snowball method of stemming. Considered a widely used method in NLP

for df_pre_processing in df_list_pre_processing: # Iterating through the list of dataframes

  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: contractions.fix(x)) # function takes a string input. Using apply with lambda to apply to series per string

  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: regex_tokenizer.tokenize(x)) # Similar issue, regex_tokenizer.tokenize() function takes a string.

  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: [snowball.stem(w) for w in x]) # List comprehension to cycle through tokens and apply contraction expansion.

  df_pre_processing['review'] = df_pre_processing['review'].apply(lambda x: ' '.join([w for w in x if not w.lower() in english_stopwords])) # Same as stemming

df_list_pre_processing[0]['review'] # Checking output

0         "it side effect , take combin bystol 5 mg fish...
1         "mi son halfway fourth week intuniv . becam co...
2         "i use take anoth oral contracept , 21 pill cy...
3         "this first time use ani form birth control . ...
4         "suboxon complet turn life around . feel healt...
                                ...                        
161292    "i wrote first report mid -octob 2014 . ani al...
161293    "i given iv befor surgey . immedi becam anxiou...
161294    "limit improv 4 month , develop bad rash md re...
161295    "i thyroid medic 49 year , spent first 38 synt...
161296    "i chronic constip adult life . tri linzess , ...
Name: review, Length: 161297, dtype: object

## Representation Learning

For the representation learning, I'm aiming to use CountVectorization() for the representation learning this time. It convert the review series into a matrix of token counts.

In [35]:
df_train = df_list_pre_processing[0].copy() # Taking copies of previous dfs, saves rerunning cells during dev
df_test = df_list_pre_processing[1].copy()

c_vect = CountVectorizer() # Instantiating the CountVectorizer object
x_train = c_vect.fit_transform(df_train['review']) # Fitting the corpus
x_test = c_vect.transform(df_test['review']) # vectorizing the test data

## NLP Model

In [36]:
decision_tree = DecisionTreeClassifier() # Instantiating the object. No parameters, don't want to use "magic numbers"
decision_tree_training_start = time.time()
decision_tree.fit(x_train, df_train['sentiment'])
decision_tree_training_finish = time.time()
decision_tree_pred = decision_tree.predict(x_test)
decision_tree_prediction_finish = time.time()



## Evaluation

In [37]:
print('Training time: ', decision_tree_training_finish - decision_tree_training_start)
print('Prediction time: ', decision_tree_prediction_finish - decision_tree_training_finish)
decision_tree_results = classification_report(df_test['sentiment'], decision_tree_pred)
print(decision_tree_results)

Training time:  310.6986277103424
Prediction time:  0.043786048889160156
              precision    recall  f1-score   support

         Bad       0.79      0.78      0.78     13497
        Good       0.93      0.93      0.93     40269

    accuracy                           0.89     53766
   macro avg       0.86      0.85      0.85     53766
weighted avg       0.89      0.89      0.89     53766



It seems the Decision tree performed better on all aspects except for "Good" recall, but the difference is negligble. Overall, more pre-processing was applied to pipeline 2 with a more complex model method. The difference between the two is 12% accuracy, however the training of this model took considerably longer. A 19275% increase in training time.

# Conclusion

Comparing the two results, Pipeline 1 had a decent overall accuracy of 77% with a training time of 1.6 seconds. Whereas Pipeline 2 had an overall accuracy of 89% with a training time of 310 seconds.

Pipeline 2, though slower, performed better. The determining factor here is Pipeline 1's abysmal performance at identifying Bad labels. This may be due to more operations implemented at pre-processing, or perhaps a better model (Though slower by approx 19275%). Pipeline 1 had the benefit of a more advanced learning representation only.

Pipeline 2 would be the pipeline of choice for the focus area.

## NLP Pipeline 1 Results


In [38]:
print('Training time: ', NB_classifier_training_finish - NB_classifier_training_start)
print('Prediction time: ', NB_classifier_training_finish - NB_classifier_prediction_finish)
NB_classifier_results = classification_report(df_test['sentiment'], NB_prediction)
print(NB_classifier_results)

Training time:  1.6072309017181396
Prediction time:  -0.02286219596862793
              precision    recall  f1-score   support

         Bad       0.92      0.09      0.17     13497
        Good       0.77      1.00      0.87     40269

    accuracy                           0.77     53766
   macro avg       0.84      0.55      0.52     53766
weighted avg       0.80      0.77      0.69     53766



## NLP Pipeline 2 Results


In [39]:
print('Training time: ', decision_tree_training_finish - decision_tree_training_start)
print('Prediction time: ', decision_tree_prediction_finish - decision_tree_training_finish)
decision_tree_results = classification_report(df_test['sentiment'], decision_tree_pred)
print(decision_tree_results)

Training time:  310.6986277103424
Prediction time:  0.043786048889160156
              precision    recall  f1-score   support

         Bad       0.79      0.78      0.78     13497
        Good       0.93      0.93      0.93     40269

    accuracy                           0.89     53766
   macro avg       0.86      0.85      0.85     53766
weighted avg       0.89      0.89      0.89     53766

